__Notes:___

- In step 5 we created CI pipeline with jobs `build`, `test` and `experiments`.
- Here add a new job `deploy` for deploying model on `merge request` event to branch `master`

# Checkout to `experiments` branch and pull last 

```bash 

git checkout experiments
git pull
```

# Add `deploy` job to CI/CD pipeline 


## Add new stage

```yaml

stages:
  - build
  - test
  - experiments
  - deploy # <- add new stage 
```


### Add `deploy` job

```yaml

deploy:
  tags:
    - shell
  only:
    refs:
      - merge_requests
    variables:
      - $CI_MERGE_REQUEST_TARGET_BRANCH_NAME == "master"
  stage: deploy
  script:
    - cd /tmp/$CI_PROJECT_NAME
    - pwd
    - |
      if ! curl http://localhost:9000/healthcheck; then
        docker run \
            -d \
            --name=deploy-sklearn-$(date +"%Y-%m-%d-%H-%M-%S") \
            -v $(pwd)/$PROJECT_MODEL_PATH:$DOCKER_MODEL_PATH \
            -e MODEL_PATH=$DOCKER_MODEL_PATH \
            -p $DEPLOY_PORT:9000 \
                $DEPLOY_IMAGE
      else
        curl -X POST http://localhost:9000/reload-model
      fi

```

# Push commits to GitLab repo

```bash
git add .gitlab-ci.yml
git commit -m "Add CI job 'deploy'"
git push origin experiments
```

# Test deployment (on local HOST) 

## Check connection


```bash 

curl http://localhost:9000/healthcheck
 
```

## Get predictions


```bash 

curl http://localhost:9000/predict \
    -d '{"data":[[value11,value12,...,value1N],...,[value1M,valueM2,...,valueMN]]}'

``` 
        
where:

M - number if rows to predict;
N - number if columns.

Columns order is the same as in train dataset.

Example:

```bash

curl -POST http://localhost:9000/predict \
     -d '{"data":[[4,3,2,1,0,0], [1,3,2,1,2,0]]}'
        
```

In [2]:
!curl http://localhost:9000/healthcheck

OK

In [7]:
%%bash 

curl -POST http://localhost:9000/predict \
     -d '{"data":[[4,3,2,1,0,0], [1,3,2,1,2,0]]}'

{
  "predictions": [
    1, 
    1
  ]
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    80  100    41  100    39   4555   4333 --:--:-- --:--:-- --:--:--  8888
